In [1]:
# Import thư viện: pandas, numpy, TransactionEncoder, fpgrowth, association_rules
import pandas as pd
import numpy as np

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [2]:
# Đọc dữ liệu và giới hạn 5000 dòng để demo
df = pd.read_excel('Online Retail.xlsx')
df = df.head(5000)

In [3]:
# Gom nhóm theo (InvoiceNo, StockCode) để chuẩn hóa mỗi hóa đơn-mỗi sản phẩm
invoice_stockcode = df.groupby(['InvoiceNo', 'StockCode']).size().reset_index(name='Count')

In [5]:
# In kích thước và xem vài dòng đầu của bảng nhóm
print(invoice_stockcode.shape)
invoice_stockcode.head()

(4849, 3)


,InvoiceNo,StockCode,Count
0,536365,21730,1
1,536365,22752,1
2,536365,71053,1
3,536365,84029E,1
4,536365,84029G,1


In [6]:
# Tính số hóa đơn khác nhau mỗi StockCode xuất hiện và xem đầu bảng
stockcode_freq = df.groupby('StockCode')['InvoiceNo'].nunique().reset_index(name='Freq')
print(stockcode_freq.shape)
stockcode_freq.head()

(1595, 2)


,StockCode,Freq
0,10002,3
1,10125,1
2,10133,1
3,10135,2
4,11001,1


In [7]:
# Tính tổng số hóa đơn và support đơn lẻ cho từng StockCode
total_invoices = df['InvoiceNo'].nunique()
stockcode_freq['Support'] = stockcode_freq['Freq'] / total_invoices
print(stockcode_freq.shape)
stockcode_freq.head()

(1595, 3)


,StockCode,Freq,Support
0,10002,3,0.010000
1,10125,1,0.003333
2,10133,1,0.003333
3,10135,2,0.006667
4,11001,1,0.003333


In [8]:
# Sắp xếp theo support giảm dần và in top 10 sản phẩm phổ biến
stockcode_freq_sorted = stockcode_freq.sort_values(by='Support', ascending=False)
print(stockcode_freq_sorted.head(10))

     StockCode  Freq   Support
922      22632    38  0.126667
1504    85123A    35  0.116667
923      22633    32  0.106667
1414    84029E    30  0.100000
1147     22961    27  0.090000
1006     22752    26  0.086667
1415    84029G    24  0.080000
1089     22866    23  0.076667
559      22086    22  0.073333
1088     22865    22  0.073333


In [9]:
# Chuẩn hóa chuỗi cho InvoiceNo/StockCode (thêm khoảng trắng)
df['InvoiceNo'] = df['InvoiceNo'].astype('str') + ' '
df['StockCode'] = df['StockCode'].astype('str') + ' '

In [10]:
# Xây danh sách transactions theo từng hóa đơn và in tổng số giao dịch
transactions = []
invoices = df['InvoiceNo'].unique()

for iv in invoices:
    items = df[df['InvoiceNo'] == iv]['StockCode'].tolist()
    transactions.append(items)

print('Total transactions: ', len(transactions))

Total transactions:  300


In [12]:
# In thử 5 giao dịch đầu để kiểm tra
print(transactions[:5])

[['85123A ', '71053 ', '84406B ', '84029G ', '84029E ', '22752 ', '21730 '], ['22633 ', '22632 '], ['84879 ', '22745 ', '22748 ', '22749 ', '22310 ', '84969 ', '22623 ', '22622 ', '21754 ', '21755 ', '21777 ', '48187 '], ['22960 ', '22913 ', '22912 ', '22914 '], ['21756 ']]


In [11]:
# Mã hóa one-hot bằng TransactionEncoder và tạo DataFrame, in kích thước
encoder = TransactionEncoder()
t_encoded = encoder.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(t_encoded, columns=encoder.columns_)
print(df_encoded.shape)
df_encoded.head()

(300, 1595)


,10002,10124G,10125,10133,10135,11001,15036,15044B,15056BL,15056N,...,90214M,90214R,90214S,90214V,BANK CHARGES,C2,D,DOT,M,POST
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
# Chạy FP-Growth với min_support=0.05 để tìm frequent itemsets
frequent_itemsets = fpgrowth(df_encoded, min_support=0.05, use_colnames=True)
print(frequent_itemsets.shape)

(630, 2)


In [14]:
# Sinh luật kết hợp từ frequent itemsets (metric=confidence, min_threshold=0.2)
fp_rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2, num_itemsets=len(frequent_itemsets))

In [15]:
# In số lượng luật tìm được
print(len(fp_rules))

13604


In [17]:
# Sắp xếp theo confidence giảm dần, in 10 luật mạnh nhất (support/confidence/lift)
fp_rules_sorted = fp_rules.sort_values(by='confidence', ascending=False)
for i, (index, rule) in enumerate(fp_rules_sorted.head(10).iterrows()):
    product_a = ', '.join(list(rule['antecedents']))
    product_b = ', '.join(list(rule['consequents']))
    print(f'{product_a} -> {product_b} (support: {rule["support"]:.3f}, confidence: {rule["confidence"]:.3f}, lift: {rule["lift"]:.3f})')

21068 , 21871 , 84029G  -> 84029E  (support: 0.050, confidence: 1.000, lift: 10.000)
82482  -> 21730 , 71053  (support: 0.050, confidence: 1.000, lift: 17.647)
21068 , 84029E , 85123A  -> 21071 , 37370  (support: 0.053, confidence: 1.000, lift: 17.647)
21068 , 84029E , 37370  -> 85123A , 21071  (support: 0.053, confidence: 1.000, lift: 17.647)
21068 , 85123A , 37370 , 21071  -> 84029E  (support: 0.053, confidence: 1.000, lift: 10.000)
21068 , 84029E , 85123A , 21071  -> 37370  (support: 0.053, confidence: 1.000, lift: 13.636)
21068 , 84029E , 37370 , 21071  -> 85123A  (support: 0.053, confidence: 1.000, lift: 8.571)
21068 , 84029E , 85123A , 37370  -> 21071  (support: 0.053, confidence: 1.000, lift: 15.000)
82482 , 71053  -> 21730  (support: 0.050, confidence: 1.000, lift: 17.647)
21068 , 85123A  -> 84029E , 37370 , 71053  (support: 0.053, confidence: 1.000, lift: 18.750)
